In [6]:
import keras
from keras.models import Model
from keras.optimizers import Adadelta
from keras.callbacks import EarlyStopping
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.preprocessing.image import load_img, array_to_img, img_to_array
from keras.optimizers import Adam
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout, UpSampling2D
import glob
from keras.callbacks import ModelCheckpoint

In [7]:
TRAIN_IMAGES = sorted(glob.glob('/kaggle/input/noise_doc/noise_doc/*.png'))
CLEAN_IMAGES = sorted(glob.glob('/kaggle/input/original_doc/original_doc/*.png'))
print("Total number of images in the training set: ", len(TRAIN_IMAGES))
print("Total number of cleaned images found: ", len(CLEAN_IMAGES))

Total number of images in the training set:  144
Total number of cleaned images found:  144


In [8]:
# Lets' define our autoencoder now
def build_autoenocder():
    input_img = Input(shape=(420,540,1), name='image_input')    
    #enoder 
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv1')(input_img)
    x = MaxPooling2D((2,2), padding='same', name='pool1')(x)   
    #decoder
    x = Conv2D(32, (3,3), activation='relu', padding='same', name='Conv2')(x)
    x = UpSampling2D((2,2), name='upsample3')(x)
    x = Conv2D(1, (3,3), activation='sigmoid', padding='same', name='Conv3')(x)   
    #model
    autoencoder = Model(inputs=input_img, outputs=x)
    autoencoder.compile(optimizer='Adam', loss='binary_crossentropy')
    return autoencoder

autoencoder = build_autoenocder()
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 420, 540, 1)       0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 420, 540, 32)      320       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 210, 270, 32)      0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 210, 270, 32)      9248      
_________________________________________________________________
upsample3 (UpSampling2D)     (None, 420, 540, 32)      0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 420, 540, 1)       289       
Total params: 9,857
Trainable params: 9,857
Non-trainable params: 0
_________________________________________________________

In [9]:
X = []
Y = []

train_images = TRAIN_IMAGES
train_labels = CLEAN_IMAGES

for img in train_images:
    img = load_img(img, grayscale=True,target_size=(420,540))
    img = img_to_array(img).astype('float32')/255.
    X.append(img)

for img in train_labels:
    img = load_img(img, grayscale=True,target_size=(420,540))
    img = img_to_array(img).astype('float32')/255.
    Y.append(img)


X = np.array(X)
Y = np.array(Y)

print("Size of X : ", X.shape)
print("Size of Y : ", Y.shape)

Size of X :  (144, 420, 540, 1)
Size of Y :  (144, 420, 540, 1)


In [18]:
autoencoder = build_autoenocder()
autoencoder.summary()

# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1)#, save_best_only=True, mode='max')
checkpoint = ModelCheckpoint("best_model.hdf5", monitor='val_acc', verbose=1,save_best_only=True, mode='auto', period=1)
#callbacks_list = [checkpoint]

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 420, 540, 1)       0         
_________________________________________________________________
Conv1 (Conv2D)               (None, 420, 540, 32)      320       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 210, 270, 32)      0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 210, 270, 32)      9248      
_________________________________________________________________
upsample3 (UpSampling2D)     (None, 420, 540, 32)      0         
_________________________________________________________________
Conv3 (Conv2D)               (None, 420, 540, 1)       289       
Total params: 9,857
Trainable params: 9,857
Non-trainable params: 0
_________________________________________________________

In [19]:
# Train your model
autoencoder.fit(X, Y, epochs=2, batch_size=16, callbacks=[checkpoint])

Epoch 1/2
144/144 [==============================] - 1s 6ms/step - loss: 0.5640

Epoch 00001: loss improved from inf to 0.56404, saving model to best_model.hdf5
Epoch 2/2
144/144 [==============================] - 1s 5ms/step - loss: 0.3489

Epoch 00002: loss improved from 0.56404 to 0.34895, saving model to best_model.hdf5


In [ ]:
# Get the predition
sample_test = load_img(train_images[100], grayscale=True, target_size=(420,540))
sample_test = img_to_array(sample_test)
sample_test_img = sample_test.astype('float32')/255.
sample_test_img = np.expand_dims(sample_test, axis=0)

# Get the predition
predicted_label = np.squeeze(autoencoder.predict(sample_test_img))

f, ax = plt.subplots(1,2, figsize=(10,8))
ax[0].imshow(np.squeeze(sample_test), cmap='gray')
ax[1].imshow(np.squeeze(predicted_label.astype('int8')), cmap='gray')
plt.show()